# DapticsClient - Introduction

This notebook contains an interactive introduction to the Python DapticsClient class,
a simplified interface for accessing the Daptics GraphQL API for the optimization of
experimental design.

Documentation for using the DapticsClient class (implemented in the daptics_client.py
file in this folder) is included as comment lines in the interactive Python cells of
this notebook.

For additional help or information, please visit or contact Daptics.

On the web at https://daptics.ai
By email at support@daptics.ai

Daptics API Version 0.7.3  
Copyright (c) 2019 Daptics Inc.

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), the rights to use, copy, modify, merge, publish, and/or distribute, copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

You do not have the right to sub-license or sell copies of the Software.

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

# Connect and start a session

## Step 1. Create a DapticsClient object and connect to the API server.

Before running this project, please make sure that your Jupyter Python environment supports Python 3, and has these required packages installed:
*   chardet
*   urllib3
*   requests
*   gql

You will also need a validated user account on the Daptics API server.  You can create an account by [registering](https:daptics.ai/register) at https://daptics.ai

See the 01_README.ipynb notebook in this folder for more information.

The following cell creates a DapticsClient object and connect to the API server.


In [ ]:
# Import the daptics_client module
# Requirements are Python 3, and the `gql` and `requests` libraries
from daptics_client import DapticsClient
from time import sleep

# Create a client object and try to connect to the daptics API server.
# The client constructor takes a URL argument that specifies the scheme,
# host and port number of the API server.

# Note: do not specify the path on the server; the DapticsClient object
# will take care of that for us.
api_host = 'http://inertia.protolife.com:8080'
daptics = DapticsClient(api_host)

# The `connect` method will attempt to connect to the /api path on the
# API server and obtain the GraphQL schema.
daptics.connect()

__Details:__  after connecting, the daptics object should have a `gql` attribute.
You can look at the data stored in the `gql` attribute by printing `daptics.gql.__dict__`.
In this data, you can see that the gql library has introspected all the GraphQL type,
query and mutation information exposed by the API.

## Step 2. Log in to the API server to obtain an access token.

The `login` method takes two string arguments, the user's `email` and `password`.

__Note:__ Use the real email address and password you used when you created
your account on the daptics.ai website.

In [ ]:
email = 'YOUR_EMAIL@YOUR_DOMAIN'
password = 'YOUR_PASSWORD'
login_data = daptics.login(email, password)

## Step 3. Create a daptics session on the server.

The `create_session` method takes three arguments:
* `name` - the name for the session (which must be unique for your account)
* `description` - a required, short description
* `is_demo` - a boolean value that determines whether the session will run
in demo mode (if `True`) or in standard mode (if `False`)

The session will be created for the `userId` saved from the login method.

Note: *each session name must be a new, unique name*. So, every time the following cellis executed, the session name must be changed.

In [ ]:
name = 'Practice Session 36'
description = 'This is a practice session'
daptics.create_session(name, description);

**Please note**: if you want to start a new session, you may execute the cell above, changing the name. It is not necessary to restart the Python kernel.

# Experimental Space Definition

## Step 4. Specify the _Experimental Space Definition_.

The _Experimental Space Definition_ (ESD) may be defined by a
CSV file that defines the names and constraints for each parameter. There are currently two different types of experiments, `mixture` and `factorial` (see tutorial notebook `02_Terminology`):

For `mixture` experiments, each row has exactly four columns:
* the parameter name (a string)
* the parameter type, set to `unit` for mixture experiments
* the minimum number of units for the parameter (an integer)
* the maximum number of units for the parameter (an integer)

For `factorial` experiments, each row has at least four columns,
but can have many more:
* the parameter name (a string)
* the parameter type, set to either `numerical` or `categorical`
* the first possible value for the parameter (a number)
* the second possible value for the parameter (a different number)
* ...etc

Each variable may have up to 20 values specified.

You should add blank padding columns for rows that have
fewer parameter values, so that the CSV file has the same number of
columns on each row.

Your ESD file should be placed in the directory that
the Jupyter notebook server process was started. The current location of this tutorial is
`../daptics-api/python_client/`; the tutorial file should appear in the `HOME` tab of the Jupyter notebook.

### Please note: running this tutorial will be free of charge for any ESDs that fit in the Daptics [Free zone](https://daptics.ai/pdt_pricing).
For this tutorial, we will use a free ESD, specified in the `esd-factorial-4x8.csv` file
installed in `../daptics-api/python_client/`.
To read your own ESD file, you would substitute your file name
for `esd-factorial-4x8.csv`.
The contents of the ESD file read as follows:


In [ ]:
fname = 'esd-factorial-4x8.csv'
with open(fname, 'r') as f:
    print(f.read())

## Step 5. Define the experimental space parameters for the session.

The `save_experimental_and_space_parameters_csv` method requires two
arguments:
* `params` - a dict of parameters that specify
    * `'space'`: a dict to define the space type, either
        * `{'type': 'factorial'}` or
        * `{'type': 'mixture', 'totalunits' : N}`, where `N` is the total number of units that all the
        mixture variables must sum to.
    * `'populationSize'`: how many experiments will be designed for each generation, and
    * `'replicates'`: how many replicates of an experiment will be performed
    (Note: to perform each experiment in the design just once, `replicates` should be zero.)
* `fname` - The path to the _Experimental Space Definition_ file discussed above.

In [ ]:
# Here is an example for a "mixture" space, which must include a
# `totalUnits` number:
#params = {
#    'space': {
#        'type': 'mixture',
#        'totalUnits': 15
#    },
#    'populationSize': 30,
#    'replicates': 2
#}
#fname = 'esd-mixture-5.csv'


# OR:  here is an example for a "factorial" space, which doesn't need the
# `totalUnits` parameter:
params = {
    'space': {
        'type': 'factorial'
    },
    'populationSize': 30,
    'replicates': 2
}
fname = 'esd-factorial-4x8.csv'

# When we call this method, the session will validate our parameters
# and if the validation succeeds, a potentially long-running process
# to set up the exploration space will be started.
task = daptics.save_experimental_and_space_parameters_csv(fname, params)

## Step 6. poll the task and process results

Execution of the cell above launches a task to validate the _Experimental Space Definition_ and set up the space.
The following call will poll this task.  This call may be repeated, ruturning `status = running` while the task is still in progress. When the task is done (`status=success`), the results are then processed, and the task is removed from the queue.  subsequent calls to `poll_for_current_task()` will result in an error because no task will be found.

**Please note**:  when the task changes to `status=success`, the results are not processed *until `poll_for_current_task()` is called*.  The session function `wait_for_current_task()` simply wraps the loop in the following cell in a function, waiting for the task to finish, and the final call to `poll_for_current_task()` that yields `status=success` also processes the results.


In [ ]:
# Poll the task. Repeat until task disappears, when `status` is `success` (or `failure`).
while True:
    task, errors = daptics.poll_for_current_task()
    if task['currentTask'] is not None:
        print("status = ", task['currentTask']['status'])
        sleep(0.5)
    else:
        print("No current task found!")
        break

# OR-----------------
# daptics.wait_for_current_task()

In [ ]:
daptics.wait_for_current_task()

## Step 7. Print session, save validated space.

If the _Experimental Space Definition_ was successfully validated, we may see it by printing out the current session state:

In [ ]:
daptics.print()

The validated _Experimental Space Definition_ object may be retrieved with the following call:

In [ ]:
space = daptics.get_validated_experimental_space()
space

and it may be exported to a CSV file (whose contents should be the same as the _Experimental Space Definition_ CSV file):

In [ ]:
fname = 'validated_space.csv'
space = daptics.export_validated_experimental_space_csv(fname)

# Show the contents of the exported file.
with open(fname, 'r') as f:
    print(f.read())

# Show the other space parameters.
# print(space)

# Initial Experiments

## Step 8. Specify initial experiments (*optional*)

Daptics modeling may be initialized with any available data from initial experiments done previously.  The data file for initial experiments must have a certain form: one row of header, followed by one row for each experiment.

### Step 8-a. Initialize with your data

The data file may be initialized with the following call:

In [ ]:
fname = 'initial_experiments.csv'
columns = daptics.export_initial_experiments_template_csv(fname)

# Print out the column names (parameter names and `Response`).
print(columns)

# Let's verify the contents of the exported file. It should just
# have one line listing the parameter names as well as a final column
# named `Response`, separated by commas.
#
# param1,param2,param3,param4,Response

with open(fname, 'r') as f:
    print(f.read())

The file `initial_experiments.csv` has been created in the Jupyter Notebook home directory (i.e. the directory where the Jupyter Notebook server was launched). You may now fill the file by adding rows to the CSV file, one row for each experiment, containing a value for each of the experimental variables (the specification of the experiment) followed by that experiment's response.

You may construct the initial experiments file from scratch, but **it must have the correct column names in the header row**.

### Step 8-b (alternative to step 8-a). Initialize with random experiments and responses:

The following cell creates random experiments with random responses, just as an example for processing initial experiments.

In [ ]:

space = daptics.get_validated_experimental_space()
design = None
num_extras = 20
random_experiments = daptics.random_experiments_with_responses(space, design, num_extras)

# Now let's save these to a file using the `export_csv` utility method.
fname = 'initial_experiments.csv'
task = daptics.export_csv(fname, random_experiments)

# Show the contents of the file we just saved.
with open(fname, 'r') as f:
    print(f.read())

## Step 9 (*run only following step 8*). Process Initial Experiments

Submit the initial experiments file created in the
previous step to create the first experimental design.



In [ ]:
fname = 'initial_experiments.csv'
task = daptics.save_initial_experiments_csv(fname)

## Step 10. Poll the task, print results

As in Step 6, the following call will poll the task launched above to process initial experiments and create the first design. This call may be repeated, ruturning status = running while the task is still in progress. When the task is done (status=success), the results are then processed, and the task is removed from the queue. Subsequent calls to poll_for_current_task() will result in an error because no task will be found.

In [ ]:
# Poll the task. Repeat until task disappears, when `status` is `success` (or `failure`).
daptics.wait_for_current_task()

In the following cell, when you print the state of the session,
you should now see the Design printed after the Experimental Space Definition.  Note that the Design printed has no responses in the final column.

In [ ]:
daptics.print()

# Process the experimental design

## Step 11. Save design to CSV
Once the current generation design is created, save it to
a CSV file.
The call `daptics.export_generated_design_csv()` saves the last generated design.

In [ ]:
# name file for current generation:
fname = 'gen'+str(daptics.gen)+'_design.csv'
# e.g. fname = 'gen1_design.csv'
print(fname)
design = daptics.export_generated_design_csv(fname)

# Show the contents of the exported file, which will contain
# the designed parameter values and a blank `Response` column
# that we will fill in after running the designed experiments.
with open(fname, 'r') as f:
    print(f.read())

# Show the design (in Python) as well.
print("Design file = ",fname)
design

## Step 12. Specify experimental responses

### Step 12-a. Specify your experimental responses

The previous cell has written the designed experiments to a file named
`genN_experiments.csv` (with N corresponding to the current generation):



In [ ]:
fname = 'gen'+str(daptics.gen)+'_design.csv'
# e.g. fname = 'gen1_design.csv'
print("Current design = ",fname)

Either using a spreadsheet or a text editor,
fill in a numerical (floating point) response value for each experiment row,
in the last entry of each row.

After saving this file, return to this notebook to
show the response file contents.

### Step 12-b (alternative to step 12-a). Specify random responses
Create random response values for the
design we just generated using the client's
`random_experiments_with_responses` method.

We need to have the experimental space and the recently
generated design for the call to `random_experiments_with_responses`.
We do not want any extra experiments, so we pass `0` for the
`num_extras` argument.

In [ ]:
space = daptics.get_validated_experimental_space()
design = daptics.get_generated_design()
num_extras = 0
random_experiments = daptics.random_experiments_with_responses(space, design, num_extras)

Now let's save these to a file using the `export_csv` utility method.

In [ ]:
fname = 'gen'+str(daptics.gen)+'_experiments.csv'
# e.g. fname = 'gen1_experiments.csv'
task = daptics.export_csv(fname, random_experiments)

# Show the contents of the file we just saved.
# with open(fname, 'r') as f:
#    print(f.read())

## Step 13.

Now save these experiments to the session.
This will start another `generate` task, just as we did
in Step 9.

In [ ]:
fname = 'gen'+str(daptics.gen)+'_experiments.csv'
# e.g. fname = 'gen1_experiments.csv'
task = daptics.save_experiment_responses_csv(fname)

In [ ]:
# Poll the task. Repeat until task disappears, when `status` is `success` (or `failure`).
daptics.wait_for_current_task()

From this point on, you can repeat steps 11 through
13 until you are satisfied with your experimental
campaign, or until all the possible experimental
parameter combinations have been exhausted.

# Full loop
The following loop illustrates how the execution of steps 1–13 can be
fully automated. This approach may be helpful, for example,
if you generate your responses by means of simulations.

In [ ]:
###################################################################
# Connect (step 1)
api_host = 'http://inertia.protolife.com:8080'
daptics = DapticsClient(api_host)
daptics.connect()
# Login (step 2) - uncomment and fill in correct information
#email = 'YOUR_EMAIL@YOUR_DOMAIN'
#password = 'YOUR_PASSWORD'
#daptics.login(email, password);

# start a session (step 3)
# NB:  session name must be changed each time this cell is executed.
name = 'Session 10'
description = 'This is a full loop session'
daptics.create_session(name, description);

###################################################################
# Experimental space:
# (steps 4-7 above)
params = {
    'space': {
        'type': 'factorial'
    },
    'populationSize': 30,
    'replicates': 2
}
fname = 'esd-factorial-4x8.csv'
# Save and validate the space:
task = daptics.save_experimental_and_space_parameters_csv(fname, params)
daptics.wait_for_current_task()
print("Experimental space done")

###################################################################
# Initial experiments:
# (steps 8-10)
space = daptics.get_validated_experimental_space()
design = None
num_extras = 20
random_experiments = daptics.random_experiments_with_responses(space, design, num_extras)
fname = 'initial_experiments.csv'
task = daptics.export_csv(fname, random_experiments)
task = daptics.save_initial_experiments_csv(fname)
daptics.wait_for_current_task()
print("Initial experiments saved.")
# Save design:
# name file for current generation:
fname = 'gen'+str(daptics.gen)+'_design.csv'
# e.g. fname = 'gen1_design.csv'
print("Saving design to: ",fname)
design = daptics.export_generated_design_csv(fname)

#######################################################################
# Loop:
# (steps 12-13 -- repeated for 5 generations)
for n in range(5):
    ###################################################################
    # Fill design with random responses
    space = daptics.get_validated_experimental_space()
    design = daptics.get_generated_design()
    num_extras = 0
    # Normally, you would fill the design with your experimental responses here.
    # For this tutorial we execute the following line
    # providing random numbers as a surrogate for the experimental responses.
    random_experiments = daptics.random_experiments_with_responses(space, design, num_extras)
    fname = 'gen'+str(daptics.gen)+'_experiments.csv'
    # e.g. fname = 'gen1_experiments.csv'
    task = daptics.export_csv(fname, random_experiments)
    # save responses and generate next design
    task = daptics.save_experiment_responses_csv(fname)
    daptics.wait_for_current_task()
    print("Done with design. Now gen = ",daptics.gen)
    fname = 'gen'+str(daptics.gen)+'_design.csv'
    # e.g. fname = 'gen1_design.csv'
    print("Saving design to: ",fname)
    design = daptics.export_generated_design_csv(fname)
print("Loop completed")